In [335]:
import pandas as pd

## Exporting the AP3

In [2]:
import json
import labelbox
import pandas as pd

In [12]:
with open('config_annotation.json') as f:
    config = json.load(f)
API_KEY = config['API_KEY']

LB_API_KEY = API_KEY
lb = labelbox.Client(api_key=LB_API_KEY)
project = lb.get_project('clgq9oadq0226070n491f0cdl')
export_url = project.export_labels()

In [13]:
import requests

response = requests.get(export_url)

if response.status_code == 200:
    with open('exported_labels_AP3.json', 'wb') as f:
        f.write(response.content)
else:
    print(response.status_code)

In [17]:
df_export_all_labels = pd.read_json('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/exported_labels_AP3.json')

In [ ]:
import pandas as pd

def transform_row(row):
    data = row
    output_data = {}
    for cls in data['classifications']:
        column = cls['title']
        if 'answer' in cls:
            output_data.setdefault(column, []).append(cls['answer']['title'])
        elif 'answers' in cls:
            output_data.setdefault(column, []).extend([a['title'] for a in cls['answers']])
    return pd.Series(output_data)

df_transformed = df_export_all_labels['Label'].apply(transform_row)


In [34]:
df_transformed

,В цьому тексті принаймні одна дегуманізаційна метафора міститься в назві українців чи України,Оберіть всі правильні відповіді: Цей образ становить небезпеку...,Цей образ...,"Чи може цей образ викликати в людей довіру, приязнь, співчуття або жаль?",Чи може цей образ мати емоції?,Оберіть всі правильні відповіді: цей образ може мати емоції...,"Оберіть всі правильні відповіді: цей образ може мати емоції, спричинені...",Оберіть всі правильні відповіді: цей образ може мати емоції чи почуття...
0,"[ні, дегуманізація не в назві]",[для фізичного виживання росіян чи Росії],"[може діяти, але не здатен планувати і оцінюва...",[ні],[так],[властиві тваринам],[зовнішніми чинниками],"[нетривалі (хвилини, години), тривалі (тижні, ..."
1,"[так, дегуманізація в назві]",[для морального розвитку росіян чи Росії],[може планувати і діяти за планом],[ні],[так],"[властиві тваринам, властиві дорослим людям]","[зовнішніми чинниками, внутрішніми переживаннями]","[нетривалі (хвилини, години), тривалі (тижні, ..."
2,"[так, дегуманізація в назві]",[для фізичного виживання росіян чи Росії],"[може діяти, але не здатен планувати і оцінюва...",[так],[так],[властиві тваринам],[зовнішніми чинниками],"[нетривалі (хвилини, години)]"
3,"[так, дегуманізація в назві]","[для морального розвитку росіян чи Росії, для ...",[може планувати і діяти за планом],[ні],[так],[властиві дорослим людям],"[зовнішніми чинниками, внутрішніми переживаннями]","[нетривалі (хвилини, години), тривалі (тижні, ..."
4,"[так, дегуманізація в назві]","[для морального розвитку росіян чи Росії, для ...",[може планувати і діяти за планом],[ні],[так],"[властиві тваринам, властиві дорослим людям]","[зовнішніми чинниками, внутрішніми переживаннями]","[нетривалі (хвилини, години), тривалі (тижні, ..."
...,...,...,...,...,...,...,...,...
1739,"[так, дегуманізація в назві]",[для фізичного виживання росіян чи Росії],[може планувати і діяти за планом],[ні],[ні],NaN,NaN,NaN
1740,"[так, дегуманізація в назві]",[для фізичного виживання росіян чи Росії],[може планувати і діяти за планом],[ні],[так],[властиві дорослим людям],"[зовнішніми чинниками, внутрішніми переживаннями]","[нетривалі (хвилини, години), тривалі (тижні, ..."
1741,"[так, дегуманізація в назві]",NaN,"[може діяти, але не здатен планувати і оцінюва...",[так],[так],[властиві тваринам],[зовнішніми чинниками],"[нетривалі (хвилини, години)]"
1742,"[так, дегуманізація в назві]",[для фізичного виживання росіян чи Росії],[може планувати і діяти за планом],[ні],[ні],NaN,NaN,NaN


In [37]:
len(df_export_all_labels) == len(df_transformed)

True

In [38]:
df_final = pd.concat([df_export_all_labels, df_transformed], axis=1)

In [48]:
df_final = df_final.drop_duplicates(subset=['DataRow ID', 'Created By'], keep='first')

In [50]:
questions = df_final.columns[-8:]
questions

Index(['В цьому тексті принаймні одна дегуманізаційна метафора міститься в назві українців чи України',
       'Оберіть всі правильні відповіді: Цей образ становить небезпеку...',
       'Цей образ...',
       'Чи може цей образ викликати в людей довіру, приязнь, співчуття або жаль?',
       'Чи може цей образ мати емоції?',
       'Оберіть всі правильні відповіді: цей образ може мати емоції...',
       'Оберіть всі правильні відповіді: цей образ може мати емоції, спричинені...',
       'Оберіть всі правильні відповіді: цей образ може мати емоції чи почуття...'],
      dtype='object')

In [52]:
df_final['Created By'].value_counts()

oleksa.marynych@gmail.com    1550
margo.arkhypova@gmail.com      49
kate.burovova@gmail.com         1
Name: Created By, dtype: int64

In [195]:
def cohen_kappa_lists(ann1, ann2):
    """Computes Cohen kappa for pair-wise annotators for list-type annotations.
    :param ann1: annotations provided by first annotator
    :type ann1: list of lists
    :param ann2: annotations provided by second annotator
    :type ann2: list of lists
    :rtype: float
    :return: Cohen kappa statistic
    """
    # Flatten the list of lists and compute the set of unique elements
    ann1_flat = [item for sublist in ann1 for item in sublist]
    ann2_flat = [item for sublist in ann2 for item in sublist]
    uniq = set(ann1_flat + ann2_flat)

    kappa_scores = []

    for item in uniq:
        count = 0
        for an1, an2 in zip(ann1, ann2):
            if item in an1 and item in an2:
                count += 1
        A = count / len(ann1)

        cnt1 = ann1_flat.count(item)
        cnt2 = ann2_flat.count(item)
        count = ((cnt1 / len(ann1)) * (cnt2 / len(ann2)))
        E = count
        if A == 0 and E == 1:
            kappa = 0
        elif A == 1 and E == 1:
            kappa = 1
        else:
            kappa = (A - E) / (1 - E) if 1 - E else 0
        kappa_scores.append(kappa)
        kappa_scores.append(kappa)

    return round(sum(kappa_scores) / len(kappa_scores), 4)

In [196]:
from sklearn.metrics import cohen_kappa_score

def calculate_cohens_kappa_lists(df, col_name):
    df_column = df[['External ID', 'Created By', col_name]]

    overlapping_samples = df_column.groupby('External ID').filter(lambda x: len(x) > 1)
    unique_sample_ids = overlapping_samples['External ID'].unique()
    rater_pairs = []
    for sample_id in unique_sample_ids:
        raters = overlapping_samples[overlapping_samples['External ID'] == sample_id]['Created By'].tolist()
        rater_pairs.append(raters)

    kappa_scores = []
    for sample_id, rater_pair in zip(unique_sample_ids, rater_pairs):
        sample_data = overlapping_samples[overlapping_samples['External ID'] == sample_id]
        rater1_labels = sample_data[sample_data['Created By'] == rater_pair[0]][col_name].tolist()
        rater2_labels = sample_data[sample_data['Created By'] == rater_pair[1]][col_name].tolist()
        kappa = cohen_kappa_lists(rater1_labels, rater2_labels)
        kappa_scores.append(kappa)
    mean_kappa_score = sum(kappa_scores) / len(kappa_scores)

    return mean_kappa_score

In [197]:
# dealing with NA values (will be counted towards agreement)
for question in questions:
    df_final[question].loc[df_final[question].isnull()] = df_final[question].loc[df_final[question].isnull()].apply(lambda x: ['no value provided'])

In [208]:
kappas = []
for question in questions:
    kappa_ = calculate_cohens_kappa_lists(df_final, question)
    kappas.append(kappa_)
    print(f'For question \n {question} \n the agreement is {kappa_} \n\n')

For question 
 В цьому тексті принаймні одна дегуманізаційна метафора міститься в назві українців чи України 
 the agreement is 0.8484848484848485 


For question 
 Оберіть всі правильні відповіді: Цей образ становить небезпеку... 
 the agreement is 0.5757575757575758 


For question 
 Цей образ... 
 the agreement is 0.7575757575757576 


For question 
 Чи може цей образ викликати в людей довіру, приязнь, співчуття або жаль? 
 the agreement is 0.7878787878787878 


For question 
 Чи може цей образ мати емоції? 
 the agreement is 0.6060606060606061 


For question 
 Оберіть всі правильні відповіді: цей образ може мати емоції... 
 the agreement is 0.4696969696969697 


For question 
 Оберіть всі правильні відповіді: цей образ може мати емоції, спричинені... 
 the agreement is 0.6666666666666666 


For question 
 Оберіть всі правильні відповіді: цей образ може мати емоції чи почуття... 
 the agreement is 0.6060606060606061 




In [209]:
import statistics

meaningfull_questions = questions[1:8]
kappas = [calculate_cohens_kappa_lists(df_final, question) for question in meaningfull_questions]
print(kappas)
print(statistics.mean(kappas))

[0.5757575757575758, 0.7575757575757576, 0.7878787878787878, 0.6060606060606061, 0.4696969696969697, 0.6666666666666666, 0.6060606060606061]
0.6385281385281385
